In [1]:
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy
import pandas as pd
import numpy as np
import yaml

from sklearn import preprocessing
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split

In [2]:
with open('config.yml', mode="r") as f:
    config = yaml.safe_load(f)

DATA_PATH = '../data/high_diamond_ranked_10min.csv'
SAVE_FOLDER = '../data/'

data = pd.read_csv(DATA_PATH)

In [3]:
# Drop unnecessary columns, explained in EDA
data.drop(columns={'redCSPerMin', 'blueCSPerMin'}, inplace=True)
data.drop(columns={'redGoldPerMin', 'blueGoldPerMin'}, inplace=True)
data.drop(columns={'redKills', 'redDeaths', 'redFirstBlood', 'redGoldDiff', 'redExperienceDiff'}, inplace=True)
data.drop(columns={'blueEliteMonsters', 'redEliteMonsters'}, inplace=True)

In [4]:
def number_of_corr_values(df, corr_threshold:int = 0.1) -> pd.DataFrame:
    # Function ranking correlation strenght between columns and blue team victories
        
    # Calculate correlation for all columns and sort by strength
    correlation = df.loc[:, df.columns != 'gameId'].corr()
    correlation_sorted = correlation.sort_values(['blueWins'], key=abs ,ascending=False)  
        
    new_index_order = ['gameId']   # always keep id 
        
    # Iterate through sorted corelation array as long as correlation is stronger than corr_threshold,
    # add those columns to array of columns to keep
    for cor in range(len(correlation_sorted['blueWins'])):
        if abs(correlation_sorted['blueWins'][cor]) < corr_threshold:
            break
        new_index_order.append(correlation_sorted['blueWins'].index[cor])
        
    # Create and return new dataframe
    new_set = df[new_index_order]
    return new_set

In [5]:
def feature_selection(df, cols:int = 16):
    # Function that returns columns recommended by filter methods
    # Uses mutual and f_regression to calculate important columns and rank them
    X_new = df.loc[:, df.columns != 'gameId']  # we are not evaluating id column
        
    # Create selectors with given type of calculation and number of output columns (cols)
    selector_mutual = SelectKBest(mutual_info_regression, k=cols)
    selector_fregresion = SelectKBest(f_regression, k=cols)

    # Fit transforms for methods - with dropna if user didn't use any NaN deletion technique 
    result_mutual = selector_mutual.fit_transform(X_new.dropna(), X_new.dropna()['blueWins'])
    result_fregresion = selector_fregresion.fit_transform(X_new.dropna(), X_new.dropna()['blueWins'])

    # Select recommended column names from results 
    chosen_names_mutual = np.append(selector_mutual.get_support(), True)
    chosen_names_fregresion = np.append(selector_fregresion.get_support(), True)
            
    # Create list of column names with unique values (mutual OR fregresion)
    values = (list(df.columns[chosen_names_mutual]))
    values_second = (list(df.columns[chosen_names_fregresion]))
    values += [item for item in values_second if item not in values]
        
    # Return dataframe containing reduced amount of columns (+id)
    return df[values]

In [6]:
if "selection_type" in config["preparation"].keys():
    # If there is no selection type, make no selection
    if config["preparation"]["selection_type"] == "correlation":
        if "correlation_threshold" in config["preparation"].keys():
            data = number_of_corr_values(data, data["preparation"]["correlation_threshold"])
        else:
            data = number_of_corr_values(data)
    
    elif config["preparation"]["selection_type"] == "feature_selection":
        if "features_amount" in config["preparation"].keys():
            data = feature_selection(data, config["preparation"]["features_amount"])
        else:
            data = feature_selection(data)
            
    else:
        pass

After selecting features, we want to split our data into test, train and validation to process them further

In [7]:
if config["preparation"]["train_split"]+config["preparation"]["test_split"]+config["preparation"]["validation_split"] != 100:
    raise Exception("Split values together have to be 100!")
    
train_split = config["preparation"]["train_split"]
test_split = config["preparation"]["test_split"]
validation_split = config["preparation"]["validation_split"]

In [8]:
# split data to train and test by config ratio
train, test = train_test_split(data, test_size=(1-train_split/100))

# further split test part to ratio with validation 
test, validation = train_test_split(test, test_size=(1/(test_split+validation_split) * validation_split))

In [9]:
train.shape

(6915, 22)

In [10]:
test.shape

(1482, 22)

In [11]:
validation.shape

(1482, 22)

In this part we want to delete outliers, as those might negatively influenece machine learning algorithm. That is why we want to delete at least the first iteration of outliers.
There is ~5% values as outliers in the first iteration, which, we consider, is reasonable price to pay for cleaner andoutlier detection more useful data.\
We are considering values further than 3x standard deviations from the mean as outliers in our preprocessing.

In [12]:
# Function to detect and delete outliers
def delete_outliers(df : pd.DataFrame) -> pd.DataFrame:
    """
    Function deletes rows containing outlier value in any of the columns and returns adjusted dataframe
    Args
        df - dataframe containing columns to check for outliers
    Returns
        DataFrame without outlier values
    """
    for cols in df.columns:    
        # Check for each column in the dataframe    
        data_frame = df[cols]
        data_mean, data_std = np.mean(data_frame), np.std(data_frame)  # Outlier > mean+3*std OR outlier < mean-3*std

        # Outliers percentage definition
        cut_off = data_std * 3
        lower, upper = data_mean - cut_off, data_mean + cut_off 

        # Identify and remove outliers
        outliers = [False if x < lower or x > upper else True for x in data_frame] 
            
        # Information for the user about deleting rows based on given column
        if outliers.count(False) > 0:
            print(f'Identified outliers: {outliers.count(False)} in column: {cols}')
        df = df[outliers]

    return df

In [13]:
# Selected columns to outlier check
# Delete outliers for train only, as it is the only "formerly" know data part
check_outliers_columns = config["preparation"]["outliers_columns"]

for col in check_outliers_columns:
    try:
        train[col] = delete_outliers(train[[col]])
        train.dropna(inplace=True)
    except KeyError:
        pass

Identified outliers: 79 in column: blueWardsDestroyed
Identified outliers: 37 in column: blueKills
Identified outliers: 53 in column: blueDeaths
Identified outliers: 48 in column: blueAssists
Identified outliers: 27 in column: blueTotalGold
Identified outliers: 42 in column: blueAvgLevel
Identified outliers: 10 in column: blueTotalExperience
Identified outliers: 12 in column: blueTotalMinionsKilled
Identified outliers: 24 in column: blueTotalJungleMinionsKilled
Identified outliers: 9 in column: blueGoldDiff
Identified outliers: 75 in column: redWardsDestroyed
Identified outliers: 44 in column: redAssists
Identified outliers: 10 in column: redTotalGold
Identified outliers: 37 in column: redAvgLevel
Identified outliers: 11 in column: redTotalExperience
Identified outliers: 22 in column: redTotalJungleMinionsKilled


In [14]:
train.shape

(6375, 22)

In [15]:
def data_scale(train, test, validation):    
    """Scale train, test and validation dataset, based on the train MinMax. 
    It means in test and validation, numbers > 1 or < 0 are also valid, because 
    we don't know about those values before"""
    scaled_columns =  list(train.columns)
    scaled_columns.remove('gameId')  # We don't want to scale gameId
        
    # Scaling with the usage of MinMaxScaler - scale all values to <0,1> range
    scaler = preprocessing.MinMaxScaler()
    train_scaler = scaler.fit(train[scaled_columns])
    
    scaled_train = pd.DataFrame(scaler.transform(train[scaled_columns]), columns = scaled_columns)
    scaled_test = pd.DataFrame(scaler.transform(test[scaled_columns]), columns = scaled_columns)
    scaled_validation = pd.DataFrame(scaler.transform(validation[scaled_columns]), columns = scaled_columns)
        
    scaled_train['gameId'] = list(train['gameId'])   # Add id to new dataframe
    scaled_test['gameId'] = list(test['gameId'])   # Add id to new dataframe
    scaled_validation['gameId'] = list(validation['gameId'])   # Add id to new dataframe
    return scaled_train, scaled_test, scaled_validation

In [16]:
train, test, validation = data_scale(train, test, validation)

In [17]:
train.drop(columns='gameId', inplace=True)
test.drop(columns='gameId', inplace=True)
validation.drop(columns='gameId', inplace=True)

In [22]:
train.describe()

,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,blueAvgLevel,...,blueTotalJungleMinionsKilled,blueGoldDiff,redWardsDestroyed,redAssists,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalJungleMinionsKilled,blueTotalMinionsKilled
count,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,...,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000,6375.000000
mean,0.300462,0.502275,0.402384,0.429613,0.356880,0.360157,0.185098,0.022118,0.460067,0.451686,...,0.495016,0.504263,0.289691,0.360296,0.017804,0.478694,0.457765,0.496787,0.499116,0.519218
std,0.187887,0.500034,0.187289,0.196564,0.207334,0.480083,0.388408,0.107297,0.172136,0.177204,...,0.164674,0.162861,0.181812,0.208788,0.097207,0.172793,0.175930,0.167170,0.165174,0.169939
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.111111,0.000000,0.266667,0.285714,0.222222,0.000000,0.000000,0.000000,0.334760,0.375000,...,0.379310,0.392695,0.111111,0.222222,0.000000,0.353620,0.375000,0.383626,0.372881,0.401639
50%,0.333333,1.000000,0.400000,0.428571,0.333333,0.000000,0.000000,0.000000,0.452745,0.500000,...,0.482759,0.505862,0.222222,0.333333,0.000000,0.467060,0.500000,0.496671,0.508475,0.524590
75%,0.444444,1.000000,0.533333,0.571429,0.500000,1.000000,0.000000,0.000000,0.573787,0.625000,...,0.586207,0.613313,0.444444,0.500000,0.000000,0.594052,0.625000,0.612969,0.593220,0.639344
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
test.describe()

,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,blueAvgLevel,...,blueTotalJungleMinionsKilled,blueGoldDiff,redWardsDestroyed,redAssists,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalJungleMinionsKilled,blueTotalMinionsKilled
count,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,...,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000
mean,0.311741,0.520243,0.418938,0.434403,0.373932,0.363023,0.199055,0.026653,0.473113,0.451754,...,0.488936,0.511804,0.306718,0.363248,0.021930,0.479168,0.450827,0.486576,0.497324,0.514895
std,0.236259,0.499759,0.202641,0.207408,0.225368,0.481034,0.399425,0.126494,0.189346,0.185832,...,0.173688,0.178751,0.238855,0.221463,0.107256,0.184049,0.192481,0.185060,0.169971,0.180443
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.010270,-0.375000,...,-0.034483,-0.075689,0.000000,0.000000,0.000000,-0.141047,-0.625000,-0.493795,-0.237288,-0.106557
25%,0.111111,0.000000,0.266667,0.285714,0.222222,0.000000,0.000000,0.000000,0.342432,0.375000,...,0.379310,0.394417,0.111111,0.222222,0.000000,0.350169,0.375000,0.370006,0.372881,0.401639
50%,0.333333,1.000000,0.400000,0.428571,0.333333,0.000000,0.000000,0.000000,0.456902,0.500000,...,0.482759,0.508096,0.333333,0.333333,0.000000,0.465868,0.500000,0.486683,0.491525,0.524590
75%,0.444444,1.000000,0.533333,0.571429,0.500000,1.000000,0.000000,0.000000,0.590659,0.625000,...,0.603448,0.631558,0.444444,0.500000,0.000000,0.596028,0.625000,0.607862,0.593220,0.639344
max,2.444444,1.000000,1.466667,1.214286,1.611111,1.000000,1.000000,1.500000,1.305416,1.125000,...,1.017241,1.110932,2.333333,1.388889,1.000000,1.190112,1.125000,1.098820,1.118644,1.057377


In [20]:
validation.describe()

,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,blueAvgLevel,...,blueTotalJungleMinionsKilled,blueGoldDiff,redWardsDestroyed,redAssists,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalJungleMinionsKilled,blueTotalMinionsKilled
count,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,...,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000,1482.000000
mean,0.311441,0.498650,0.412236,0.441537,0.372057,0.363698,0.189609,0.025304,0.466203,0.444332,...,0.493008,0.502015,0.286100,0.374794,0.025641,0.488841,0.450574,0.492412,0.494396,0.513191
std,0.247670,0.500167,0.202083,0.207284,0.228323,0.481225,0.392124,0.125433,0.191701,0.195214,...,0.165151,0.185930,0.223497,0.230022,0.121951,0.192686,0.195436,0.184995,0.168533,0.181252
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.083140,-1.000000,...,-0.068966,-0.180466,0.000000,0.000000,0.000000,-0.159242,-0.875000,-0.619855,-0.186441,-0.278689
25%,0.111111,0.000000,0.266667,0.285714,0.222222,0.000000,0.000000,0.000000,0.331978,0.375000,...,0.379310,0.389215,0.111111,0.222222,0.000000,0.350671,0.375000,0.385480,0.372881,0.401639
50%,0.333333,0.000000,0.400000,0.428571,0.333333,0.000000,0.000000,0.000000,0.452439,0.500000,...,0.482759,0.501246,0.222222,0.333333,0.000000,0.481114,0.500000,0.495157,0.491525,0.524590
75%,0.444444,1.000000,0.533333,0.571429,0.500000,1.000000,0.000000,0.000000,0.586441,0.625000,...,0.586207,0.620879,0.444444,0.500000,0.000000,0.600452,0.625000,0.608202,0.605932,0.639344
max,2.444444,1.000000,1.266667,1.357143,1.388889,1.000000,1.000000,1.500000,1.347231,1.125000,...,1.051724,1.153356,2.222222,1.444444,1.000000,1.261764,1.250000,1.142857,1.186441,1.024590


In [21]:
train.to_csv(f"{SAVE_FOLDER}/train.csv")
test.to_csv(f"{SAVE_FOLDER}/test.csv")
validation.to_csv(f"{SAVE_FOLDER}/validation.csv")